<a href="https://colab.research.google.com/github/Eduardo-Mati/Projetos-integrador---Desafio-unifacisa---Modelos-de-Machine-Learning/blob/main/Q8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalação Básica
!pip install numpy==1.26.0 pandas

# Filtragem Colaborativa
!pip install scikit-surprise

# Aprendizado Profundo (Autoencoders)
!pip install tensorflow keras
# Opcional, mas útil:
!pip install tensorflow-recommenders
import pandas as pd
import numpy as np

from surprise.model_selection import train_test_split as surprise_split

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 3.5 MB/s eta 0:00:00


In [ ]:
# Criando dataframe
df = pd.read_csv('avaliacoes_filmes.csv')

In [ ]:
df.head(10)

,user_id,movie_id,rating,timestamp
0,1,101,5,2025-09-01
1,1,102,3,2025-09-02
2,1,103,4,2025-09-03
3,2,101,4,2025-09-01
4,2,102,2,2025-09-02
5,2,104,5,2025-09-03
6,3,101,2,2025-09-01
7,3,103,5,2025-09-02
8,3,104,3,2025-09-03
9,4,102,4,2025-09-01


In [ ]:
# --- Modelo de Filtragem Colaborativa Baseado em Usuário (User-Based) ---
print("\n--- Avaliando Modelo User-Based ---")

# Um Reader é necessário para parsear o arquivo ou dataframe
# O formato do dataframe deve ser user item rating
from surprise import Reader, Dataset, accuracy
from surprise.model_selection import train_test_split as surprise_split
from surprise.prediction_algorithms import KNNBasic

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'movie_id', 'rating']], reader)

# Divide o dataset em treino e teste usando a função do surprise
trainset, testset = surprise_split(data, test_size=.25, random_state=42)


# Configura o modelo para usar a similaridade de cossenos entre usuários
sim_options_user = {'name': 'cosine',
                    'user_based': True  # Importante: define como user-based
                   }
model_user = KNNBasic(sim_options=sim_options_user)

# Treina o modelo
model_user.fit(trainset)

# Faz as previsões no conjunto de teste
predictions_user = model_user.test(testset)

# Calcula e exibe o RMSE e MAE
print("Resultados para User-Based:")
accuracy.rmse(predictions_user)
accuracy.mae(predictions_user)


# --- Modelo de Filtragem Colaborativa Baseado em Item (Item-Based) ---
print("\n--- Avaliando Modelo Item-Based ---")

# Configura o modelo para usar a similaridade de cossenos entre itens
sim_options_item = {'name': 'cosine',
                    'user_based': False  # Importante: define como item-based
                   }
model_item = KNNBasic(sim_options=sim_options_item)

# Treina o modelo
model_item.fit(trainset)

# Faz as previsões no conjunto de teste
predictions_item = model_item.test(testset)

# Faz as previsões no conjunto de teste
predictions_item = model_item.test(testset)

# Calcula e exibe o RMSE e MAE
print("Resultados para Item-Based:")
accuracy.rmse(predictions_item)
accuracy.mae(predictions_item)


--- Avaliando Modelo User-Based ---
Computing the cosine similarity matrix...
Done computing similarity matrix.
Resultados para User-Based:
RMSE: 1.7159
MAE:  1.3333

--- Avaliando Modelo Item-Based ---
Computing the cosine similarity matrix...
Done computing similarity matrix.
Resultados para Item-Based:
RMSE: 1.8875
MAE:  1.6250


1.625

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


# Assumindo que seu DataFrame original se chama 'df_ratings'
# com colunas ['user_id', 'movie_id', 'rating']

# PRIMEIRO, VAMOS DIVIDIR OS DADOS EM TREINO E TESTE ANTES DE TUDO
# Isso garante que o teste seja feito em dados que o modelo NUNCA viu.
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# AGORA, CRIAMOS A MATRIZ USUÁRIO-ITEM APENAS COM OS DADOS DE TREINO
print("Criando a matriz usuário-item de treino...")
user_item_matrix_train = train_data.pivot_table(index='user_id', columns='movie_id', values='rating').fillna(0)

# Para a avaliação, vamos precisar dos dados de teste no mesmo formato de matriz
user_item_matrix_test = test_data.pivot_table(index='user_id', columns='movie_id', values='rating').fillna(0)

# Normalizar os dados para a rede neural aprender melhor (opcional, mas recomendado)
# A rede funciona melhor com valores pequenos, entre 0 e 1.
scaler = MinMaxScaler()
user_item_matrix_train_scaled = scaler.fit_transform(user_item_matrix_train.values)

print("Dimensões da matriz de treino:", user_item_matrix_train_scaled.shape)

Criando a matriz usuário-item de treino...
Dimensões da matriz de treino: (5, 4)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam


# A dimensão de entrada é o número de filmes
n_movies = user_item_matrix_train_scaled.shape[1]

# --- Arquitetura da Rede ---

# Camada de Entrada: tem o mesmo número de neurônios que o número de filmes
input_layer = Input(shape=(n_movies,))

# Encoder: comprime a informação
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(64, activation='relu')(encoded)

# Camada Latente (o "gargalo"): a representação mais comprimida do gosto do usuário
latent_view = Dense(32, activation='relu')(encoded)

# Decoder: tenta reconstruir a informação original a partir da camada latente
decoded = Dense(64, activation='relu')(latent_view)
decoded = Dense(128, activation='relu')(decoded)

# Camada de Saída: tem o mesmo número de neurônios da entrada, para gerar as previsões
output_layer = Dense(n_movies, activation='sigmoid')(decoded) # Sigmoid para garantir saída entre 0 e 1

# Compilando o modelo
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Mostra um resumo da arquitetura
autoencoder.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,924 (85.64 KB)

 Trainable params: 21,924 (85.64 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:

from tensorflow.keras.metrics import RootMeanSquaredError


# Dividir os dados em treino e teste (da imagem 1)
# Isso garante que o teste seja feito em dados que o modelo NUNCA viu.
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Criar a matriz usuário-item APENAS COM OS DADOS DE TREINO
user_item_matrix_train = train_data.pivot_table(index='user_id', columns='movie_id', values='rating').fillna(0)

# Criar a matriz de teste
# Precisamos garantir que ela tenha as mesmas linhas (usuários) e colunas (filmes)
user_item_matrix_test_raw = test_data.pivot_table(index='user_id', columns='movie_id', values='rating').fillna(0)

# *** CORREÇÃO IMPORTANTE: Alinhar as matrizes ***
# Garante que as matrizes de teste e treino tenham as mesmas dimensões
# O teste terá as mesmas colunas (filmes) e linhas (usuários) do treino.
user_item_matrix_test = user_item_matrix_test_raw.reindex(
    index=user_item_matrix_train.index,
    columns=user_item_matrix_train.columns
).fillna(0)


# Normalizar os dados (da imagem 1)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(user_item_matrix_train)

# *** CORREÇÃO IMPORTANTE: Usar o MESMO scaler no teste ***
# Usamos .transform() (e não .fit_transform()) para aplicar a mesma escala do treino
X_test_scaled = scaler.transform(user_item_matrix_test)

print(f"Dimensões da matriz de treino: {X_train_scaled.shape}")
print(f"Dimensões da matriz de teste: {X_test_scaled.shape}")


# --- 2. Definir o Modelo Autoencoder (Baseado na sua imagem 3) ---

# A dimensão de entrada é o número de filmes
n_movies = X_train_scaled.shape[1]

# --- Arquitetura da Rede ---
input_layer = Input(shape=(n_movies,))
# Encoder
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(64, activation='relu')(encoded)
# Camada Latente ("gargalo")
latent_view = Dense(32, activation='relu')(encoded)
# Decoder
decoded = Dense(64, activation='relu')(latent_view)
decoded = Dense(128, activation='relu')(decoded)
# Camada de Saída
output_layer = Dense(n_movies, activation='sigmoid')(decoded) # Sigmoid é bom para dados normalizados entre 0 e 1

# Compilando o modelo
autoencoder = Model(input_layer, output_layer)

# Adicionamos 'mae' e 'rmse' para monitoramento
autoencoder.compile(optimizer=Adam(learning_rate=0.001),
                    loss='mean_squared_error',
                    metrics=['mae', RootMeanSquaredError(name='rmse')])

autoencoder.summary()


# --- 3. Treinar o Modelo (A parte que faltava) ---

print("\n--- Iniciando o Treinamento do Autoencoder ---")

# AQUI ESTÁ A RESPOSTA PARA "X e y":
# x = X_train_scaled (entradas)
# y = X_train_scaled (saídas esperadas)
#
# O validation_data segue a mesma lógica:
# x = X_test_scaled
# y = X_test_scaled
history = autoencoder.fit(
    X_train_scaled,  # X (Entrada)
    X_train_scaled,  # y (Saída esperada)
    epochs=50,       # Ajuste o número de épocas conforme necessário
    batch_size=32,   # Ajuste o tamanho do lote
    shuffle=True,
    validation_data=(
        X_test_scaled, # X de validação
        X_test_scaled  # y de validação
    )
)

print("\n--- Treinamento Concluído ---")


# --- 4. Avaliar o Desempenho (A parte que faltava) ---

# Avalia o modelo final no conjunto de teste
results = autoencoder.evaluate(X_test_scaled, X_test_scaled)

print("\n--- Resultados da Avaliação no Teste ---")
# 'results' contém [loss (MSE), mae, rmse]
print(f"Perda (MSE): {results[0]:.4f}")
print(f"MAE:         {results[1]:.4f}")
print(f"RMSE:        {results[2]:.4f}")

Dimensões da matriz de treino: (5, 4)
Dimensões da matriz de teste: (5, 4)


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 128)            │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,924 (85.64 KB)

 Trainable params: 21,924 (85.64 KB)

 Non-trainable params: 0 (0.00 B)


--- Iniciando o Treinamento do Autoencoder ---
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.1667 - mae: 0.3706 - rmse: 0.4083 - val_loss: 0.2338 - val_mae: 0.4800 - val_rmse: 0.4835
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - loss: 0.1649 - mae: 0.3685 - rmse: 0.4061 - val_loss: 0.2338 - val_mae: 0.4800 - val_rmse: 0.4835
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - loss: 0.1633 - mae: 0.3666 - rmse: 0.4041 - val_loss: 0.2338 - val_mae: 0.4801 - val_rmse: 0.4835
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - loss: 0.1619 - mae: 0.3650 - rmse: 0.4024 - val_loss: 0.2338 - val_mae: 0.4802 - val_rmse: 0.4836
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - loss: 0.1604 - mae: 0.3632 - rmse: 0.4006 - val_loss: 0.2338 - val_mae: 0.4802 - val_rmse: 0.4836
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - loss: 0.1589 - mae: 0.3614 - rmse: 0.3986 - val_loss: 0.2338 - val_mae: 0.4802 - val_rmse: 0.4835
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - los

Pergunta: Quais desafios foram encontrados no treinamento? Como melhorar o desempenho do modelo?


---

Esparsidade dos Dados.

Utilizar Fatoração Matricial Avançada: Aplicar algoritmos como SVD (Singular Value Decomposition), que são explicitamente projetados para lidar com matrizes esparsas e aprender "features latentes" (gostos implícitos) dos usuários.